<a href="https://colab.research.google.com/github/namespaiva/veraoUSP-ICDMQ/blob/main/atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tratamento

In [2]:
import pandas as pd
import numpy as np
import os

# Os arquivos tiveram alterações feitas fora deste .ipynb visto que
# por erros de formatação nos dados, o pandas não conseguia fazer a leitura deles.

if 'Retail_Data.csv' not in os.listdir():
  !wget 'https://raw.githubusercontent.com/namespaiva/veraoUSP-ICDMQ/refs/heads/main/Retail_Data.csv'
  train = pd.read_csv('Retail_Data.csv')

if 'Potential_Customers.csv' not in os.listdir():
  !wget 'https://raw.githubusercontent.com/namespaiva/veraoUSP-ICDMQ/refs/heads/main/Potential_Customers.csv'
  test = pd.read_csv('Potential_Customers.csv')

In [39]:
train.head(2)

,Cocunut,Mortgage_YN,AGE_AT_ORIGINATION,AGE,YEARS_WITH_BANK,MARTIAL_STATUS,EDUCATION,EMPLOYMENT,GENDER,CUST_INCOME,CURRENT_ADDRESS_DATE,CURRENT_JOB_DATE,CURRENT_WITH_BANK_DATE,CURRENT_BALANCE_EUR
0,1,Y,50.0,52,13,Casado,Ensino Médio,PVE,Masculino,909,2004-03-16,1979-04-18,2010-01-01,5013077
1,9,Y,48.0,49,11,Casado,Ensino Médio,SFE,Masculino,288,2005-11-07,1999-12-16,2004-01-20,4615385


In [40]:
train.shape

(23983, 14)

In [31]:
train['CURRENT_JOB_DATE'] = pd.to_datetime(train['CURRENT_JOB_DATE'], errors='coerce')
temp = train['CURRENT_ADDRESS_DATE']
train['CURRENT_ADDRESS_DATE'] = pd.to_datetime(train['CURRENT_BALANCE_EUR'], errors='coerce')
train['CURRENT_BALANCE_EUR'] = temp

In [36]:
train['MARTIAL_STATUS'] = train['MARTIAL_STATUS'].replace({'S': 'Solteiro', 'M': 'Casado', 'W': 'Viúvo(a)',
                                                           'D': 'Divorciado', '*noval*': ''})
train['MARTIAL_STATUS'] = train['MARTIAL_STATUS'][train['MARTIAL_STATUS'] != '' ]
train['GENDER'] = train['GENDER'].replace({'M': 'Masculino', 'F': 'Feminino'})
train['GENDER'] = train['GENDER'][train['GENDER'] != '']
train['EDUCATION'] = train['EDUCATION'].replace({ 'HGH': 'Ensino Médio', 'BCR': 'Graduação', 'MAS': 'Mestrado',
                                                 'SEC': 'Ensino Fundamental II', 'PHD': 'Doutorado', 'PRS': 'Curso Profissionalizante',
                                                  'PRI': 'Ensino Fundamental I', 'OTH': 'Outros' })


In [37]:
train.head(2)

,Cocunut,Mortgage_YN,AGE_AT_ORIGINATION,AGE,YEARS_WITH_BANK,MARTIAL_STATUS,EDUCATION,EMPLOYMENT,GENDER,CUST_INCOME,CURRENT_ADDRESS_DATE,CURRENT_JOB_DATE,CURRENT_WITH_BANK_DATE,CURRENT_BALANCE_EUR
0,1,Y,50.0,52,13,Casado,Ensino Médio,PVE,Masculino,909,2004-03-16,1979-04-18,2010-01-01,5013077
1,9,Y,48.0,49,11,Casado,Ensino Médio,SFE,Masculino,288,2005-11-07,1999-12-16,2004-01-20,4615385


In [41]:
train.shape

(23983, 14)

# 1

- Qual é o significado exato das colunas e seus valores?
- Algumas colunas possuem inconsistências, algumas não possuem nome ou estão com valores incongruentes ao que deveriam representar, como preferem prosseguir?
- A coluna CURRENT_ADDRESS_DATE não apresenta valores de data, mas a coluna CURRENT_BALANCE_EUR, sim. Elas poderiam estar invertidas?
- Quais critérios atualmente são analisados para os empréstimos e como é feita essa análise?

# 2

# 3

# 4